In [104]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import string

#Text pre-processing

In [105]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [106]:
!cp gdrive/MyDrive/GitHub/NLP-project-/preprocessing_functions.py .

In [107]:
!python preprocessing_functions.py 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [108]:
execfile('preprocessing_functions.py')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [109]:
#df = pd.read_csv('NLP_project_kickoff_data.csv', encoding='utf-8')
df = pd.read_csv('NLP_project_kickoff_data.csv', encoding='utf-8')

In [111]:
df_emo= pd.read_csv('emojis.csv')

##Emoji replacement 

In [112]:
string_columns=['biography', 'caption', 'tokens']

In [113]:

for i in string_columns:
  emojis_replacement(df[i])  


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


##Creation of Hashtags

In [114]:
df["hashtags"] = df.caption.apply(lambda x: [x for x in x.split(" ") if x.startswith("#")])
#df['hashtags']

##Text Lower

In [115]:
df.apply(lambda x: x.astype(str).str.lower())
df

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,hashtags
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,nan,2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ...","[#breastaugmentation, #BreastEnlargemnet, #vas..."
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa...","[#rhinoplasty, #facelift, #london, #miltonkeyn..."
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipStm \r\n""Li...",British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',...","[#permalip, #lips, #cosmeticsurgery, #lipinjec..."
3,3,british_face_clinic,A recent successful Rhinoplasty procedure resu...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['White', 'Grey', 'text', 'screenshot', 'dog',...","[#rhinoplasty, #rhinoplastybeforeandafter, #su..."
4,4,dr_danmarsh,Tummytuck at 6 months post op. Final results t...,Dr Dan Marsh PhD FRCS (plast),"Jglobe4 London, UK\r\nNbuilding6 Co-founder ...",2.400000e+18,109,2778,309,72047,1,1600373084,j_kvihwg0g1c283e3byk.d165e082aac6a78850c61cecd...,https://www.instagram.com/p/CFQBpv2n2sG/,161,0,NaN,2021-11-02T19:40:32.890Z,0.144270,0.989714,0.001252,0,0,0,"['Brown', 'Black', 'Pink', 'undergarment', 'cl...",[#bodybydan]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,viciocassaro,wow.,Vincenzo Cassaro,Lcomputer Liphone Lmusic8 Lgame2 âš½â˜•ðŸ»ðŸ•,8.710000e+17,407,27,36,431,0,1418073537,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:39:59.077Z,0.001229,0.002201,0.000616,0,0,0,"['Grey', 'Black', 'outdoor', 'aerial', 'photog...",[]
515,1,viciocassaro,Fshirttie,Vincenzo Cassaro,Lcomputer Liphone Lmusic8 Lgame2 âš½â˜•ðŸ»ðŸ•,2.480000e+18,407,87,36,431,0,1610199243,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/CJ03kIkj8cH/,0,0,NaN,2021-11-06T17:39:59.077Z,0.008394,0.011220,0.000874,0,0,0,"['Black', 'person', 'christmas', 'tree', 'clot...",[]
516,2,viciocassaro,#OneStepCloser,Vincenzo Cassaro,Lcomputer Liphone Lmusic8 Lgame2 âš½â˜•ðŸ»ðŸ•,1.030000e+18,407,48,36,431,0,1436953461,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/5JvdTIJmWU/,0,0,NaN,2021-11-06T17:39:59.077Z,0.000879,0.001615,0.000563,0,0,0,"['White', 'human', 'face', 'person', 'clothing...",[#OneStepCloser]
517,0,viciocassaro,wow.,Vincenzo Cassaro,Lcomputer Liphone Lmusic8 Lgame2 âš½â˜•ðŸ»ðŸ•,8.710000e+17,407,27,36,437,0,1418073537,j_kvo3dl7njv41lgn55.c75281bab6c50a2a8905d2be3a...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:40:39.623Z,0.001229,0.002201,0.000616,0,0,0,"['Grey', 'Black', 'outdoor', 'aerial', 'photog...",[]


##Remove punctuation

In [116]:
df["biography_wo_punc"] = df["biography"].str.replace('[{}]'.format(string.punctuation), '')
df["caption_wo_punc"] = df["caption"].str.replace('[{}]'.format(string.punctuation), '')
df["profile_wo_punc"] = df["profile_name"].str.replace('[{}]'.format(string.punctuation), '')
df["tokens_wo_punc"] = df["tokens"].str.replace('[{}]'.format(string.punctuation), '')

df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,tokens_wo_punc
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,nan,2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ...","[#breastaugmentation, #BreastEnlargemnet, #vas...",nan,I am Dr Hazem Alfie Plastic and Cosmetic Surge...,NaN,White human face person smile clothing headsho...
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa...","[#rhinoplasty, #facelift, #london, #miltonkeyn...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,British Face Clinic is offering free facial co...,British Face Clinic,White Grey text human face fashion accessory s...
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipStm \r\n""Li...",British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',...","[#permalip, #lips, #cosmeticsurgery, #lipinjec...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,Need fuller lips Consider PermaLipStm \r\nLip ...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...
3,3,british_face_clinic,A recent successful Rhinoplasty procedure resu...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['White', 'Grey', 'text', 'screenshot', 'dog',...","[#rhinoplasty, #rhinoplastybeforeandafter, #su...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,A recent successful Rhinoplasty procedure resu...,British Face Clinic,White Grey text screenshot dog text text
4,4,dr_danmarsh,Tummytuck at 6 months post op. Final results t...,Dr Dan Marsh PhD FRCS (plast),"Jglobe4 London, UK\r\nNbuilding6 Co-founder ...",2.400000e+18,109,2778,309,72047,1,1600373084,j_kvihwg0g1c283e3byk.d165e082aac6a78850c61cecd...,https://www.instagram.com/p/CFQBpv2n2sG/,161,0,NaN,2021-11-02T19:40:32.890Z,0.144270,0.989714,0.001252,0,0,0,"['Brown', 'Black', 'Pink', 'undergarment', 'cl...",[#bodybydan],Jglobe4 London UK\r\nNbuilding6 Cofounder th...,Tummytuck at 6 months post op Final results ta...,Dr Dan Marsh PhD FRCS plast,Brown Black Pink undergarment clothing stomach...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,viciocassaro,wow.,Vincenzo Cassaro,Lcomputer Liphone Lmusic8 Lgame2 âš½â˜•ðŸ»ðŸ•,8.710000e+17,407,27,36,431,0,1418073537,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:39:59.077Z,0.001229,0.002201,0.000616,0,0,0,"['Grey', 'Black', 'outdoor', 'aerial', 'photog...",[],Lcomputer Liphone Lmusic8 Lgame2 âš½â˜•ðŸ»ðŸ•,wow,Vincenzo Cassaro,Grey Black outdoor aerial photography cloud wa...
515,1,viciocassaro,Fshirttie,Vincenzo Cassaro,Lcomputer Liphone Lmusic8 Lgame2 âš½â˜•ðŸ»ðŸ•,2.480000e+18,407,87,36,431,0,1610199243,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,

In [117]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [118]:
df["biography_wo_stop"] = df["biography_wo_punc"].apply(lambda text: remove_stopwords(text))
df["caption_wo_stop"] = df["caption_wo_punc"].apply(lambda text: remove_stopwords(text))
df["profile_wo_stop"] = df["profile_wo_punc"].apply(lambda text: remove_stopwords(text))
df["tokens_wo_stop"] = df["tokens_wo_punc"].apply(lambda text: remove_stopwords(text))


##Removal of frequent words

In [119]:
from collections import Counter
cnt = Counter()
for text in df["biography_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('based', 125),
 ('Enhancing', 104),
 ('ð\x8f—¡ð\x8f—®ð\x8f˜\x8fð\x8f˜‚ð\x8f—¿ð\x8f—®ð\x8f—¹', 104),
 ('ð\x8f—•ð\x8f—²ð\x8f—®ð\x8f˜‚ð\x8f˜\x8fð\x8f˜†', 104),
 ('ðŸŒ¥', 104),
 ('officialnavyprofessional', 104),
 ('Educator', 104),
 ('discount', 104),
 ('AUD10', 104),
 ('hdbrowsofficial', 104)]

In [120]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
# def remove_freqwords(text):
#     """custom function to remove the frequent words"""
#     return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["biography_wo_stopfreq"] = df["biography_wo_stop"].apply(lambda text: remove_freqwords(text))
df["caption_wo_stopfreq"] = df["caption_wo_stop"].apply(lambda text: remove_freqwords(text))
df["profile_wo_stopfreq"] = df["profile_wo_stop"].apply(lambda text: remove_freqwords(text))
df["tokens_wo_stopfreq"] = df["tokens_wo_stop"].apply(lambda text: remove_freqwords(text))

df.head()

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,tokens_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,tokens_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq,tokens_wo_stopfreq
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,nan,2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ...","[#breastaugmentation, #BreastEnlargemnet, #vas...",nan,I am Dr Hazem Alfie Plastic and Cosmetic Surge...,NaN,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon base...,nan,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon UKâ€...,nan,White human face person smile clothing headsho...
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa...","[#rhinoplasty, #facelift, #london, #miltonkeyn...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,British Face Clinic is offering free facial co...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipStm \r\n""Li...",British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',...","[#permalip, #lips, #cosmeticsurgery, #lipinjec...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,Need fuller lips Consider PermaLipStm \r\nLip ...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider PermaLipStm Lip Impl...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider PermaLipStm Lip Impl...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...
3,3,british_face_clinic,A recent successful Rhinoplasty procedure resu...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['White', 'Grey', 'text', 'screenshot', 'dog',...","[#rhinoplasty, #rhinoplastybeforeandafter, #su...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,A recent successful Rhinoplasty procedure resu...,British Face Clinic,White Grey text screenshot dog text text,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,A recent successful Rhinoplasty procedure resu...,British Face Clinic,White Grey text screenshot dog text text,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,A recent successful Rhinoplasty procedure resu...,British Face Clinic,White Grey text screenshot dog text

##removal of rare words

In [121]:

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])



In [122]:
df["biography_wo_stopfreqrare"] = df["biography_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["caption_wo_stopfreqrare"] = df["caption_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["profile_wo_stopfreqrare"] = df["profile_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["tokens_wo_stopfreqrare"] = df["tokens_wo_stopfreq"].apply(lambda text: remove_rarewords(text))

df.head()

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,tokens_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,tokens_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq,tokens_wo_stopfreq,biography_wo_stopfreqrare,caption_wo_stopfreqrare,profile_wo_stopfreqrare,tokens_wo_stopfreqrare
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,nan,2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ...","[#breastaugmentation, #BreastEnlargemnet, #vas...",nan,I am Dr Hazem Alfie Plastic and Cosmetic Surge...,NaN,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon base...,nan,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon UKâ€...,nan,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon UKâ€...,nan,White human face person smile clothing headsho...
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa...","[#rhinoplasty, #facelift, #london, #miltonkeyn...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,British Face Clinic is offering free facial co...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipStm \r\n""Li...",British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',...","[#permalip, #lips, #cosmeticsurgery, #lipinjec...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,Need fuller lips Consider PermaLipStm \r\nLip ...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider PermaLipStm Lip Impl...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider PermaLipStm Lip Impl...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider PermaLipStm Lip Impl...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...
3,3,british_face_clinic,A recent successful Rhinoplasty procedure resu...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['White', 'Grey', 'text', 'screensh

##Stemming

In [123]:
# from nltk.stem.porter import PorterStemmer

# stemmer = PorterStemmer()
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])




In [124]:
df["biography_stemmed"] = df["biography_wo_stopfreqrare"].apply(lambda text: stem_words(text))
df["caption_stemmed"] = df["caption_wo_stopfreqrare"].apply(lambda text: stem_words(text))
#df["profile_stemmed"] = df["profile_wo_punc"].str.apply(lambda text: stem_words(text))
df["tokens_stemmed"] = df["tokens_wo_punc"].apply(lambda text: stem_words(text))


##Lemmatization

In [125]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [126]:
from nltk.stem import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()
# def lemmatize_words(text):
#     return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["biography_final"] = df["biography_stemmed"].apply(lambda text: lemmatize_words(text))
df["caption_final"] = df["caption_stemmed"].apply(lambda text: lemmatize_words(text))
df["profile_final"] = df["profile_wo_stopfreqrare"].apply(lambda text: lemmatize_words(text))
df["tokens_final"] = df["tokens_stemmed"].apply(lambda text: lemmatize_words(text))

df.head()

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,tokens_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,tokens_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq,tokens_wo_stopfreq,biography_wo_stopfreqrare,caption_wo_stopfreqrare,profile_wo_stopfreqrare,tokens_wo_stopfreqrare,biography_stemmed,caption_stemmed,tokens_stemmed,biography_final,caption_final,profile_final,tokens_final
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,nan,2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ...","[#breastaugmentation, #BreastEnlargemnet, #vas...",nan,I am Dr Hazem Alfie Plastic and Cosmetic Surge...,NaN,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon base...,nan,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon UKâ€...,nan,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon UKâ€...,nan,White human face person smile clothing headsho...,nan,I Dr hazem alfi plastic cosmet surgeon ukâ€¦â€...,white human face person smile cloth headshot f...,nan,I Dr hazem alfi plastic cosmet surgeon ukâ€¦â€...,nan,white human face person smile cloth headshot f...
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa...","[#rhinoplasty, #facelift, #london, #miltonkeyn...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,British Face Clinic is offering free facial co...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,facial cosmet ent thyroid clinic Mr mrinal sup...,british face clinic offer free facial cosmet a...,white grey text human face fashion accessori s...,facial cosmet ent thyroid clinic Mr mrinal sup...,british face clinic offer free facial cosmet a...,British Face Clinic,white grey text human face fashion accessori s...
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipStm \r\n""Li...",British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',...","[#permalip, #lips, #cosmeticsurgery, #lipinjec...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,Need fuller lips Consider PermaLipStm \r\nLip ...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider PermaLipStm Lip Impl...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider Pe

##Removal of URLs

In [127]:
df['biography_final'] = df['biography_final'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df['caption_final'] = df['caption_final'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,tokens_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,tokens_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq,tokens_wo_stopfreq,biography_wo_stopfreqrare,caption_wo_stopfreqrare,profile_wo_stopfreqrare,tokens_wo_stopfreqrare,biography_stemmed,caption_stemmed,tokens_stemmed,biography_final,caption_final,profile_final,tokens_final
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,nan,2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ...","[#breastaugmentation, #BreastEnlargemnet, #vas...",nan,I am Dr Hazem Alfie Plastic and Cosmetic Surge...,NaN,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon base...,nan,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon UKâ€...,nan,White human face person smile clothing headsho...,nan,I Dr Hazem Alfie Plastic Cosmetic Surgeon UKâ€...,nan,White human face person smile clothing headsho...,nan,I Dr hazem alfi plastic cosmet surgeon ukâ€¦â€...,white human face person smile cloth headshot f...,nan,I Dr hazem alfi plastic cosmet surgeon ukâ€¦â€...,nan,white human face person smile cloth headshot f...
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa...","[#rhinoplasty, #facelift, #london, #miltonkeyn...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,British Face Clinic is offering free facial co...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,British Face Clinic offering free facial cosme...,British Face Clinic,White Grey text human face fashion accessory s...,facial cosmet ent thyroid clinic Mr mrinal sup...,british face clinic offer free facial cosmet a...,white grey text human face fashion accessori s...,facial cosmet ent thyroid clinic Mr mrinal sup...,british face clinic offer free facial cosmet a...,British Face Clinic,white grey text human face fashion accessori s...
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipStm \r\n""Li...",British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',...","[#permalip, #lips, #cosmeticsurgery, #lipinjec...",Facial Cosmetics ENT and Thyroid Clinic by Mr ...,Need fuller lips Consider PermaLipStm \r\nLip ...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider PermaLipStm Lip Impl...,British Face Clinic,White skin lip closeup eyelash lipstick organ ...,Facial Cosmetics ENT Thyroid Clinic Mr Mrinal ...,Need fuller lips Consider Pe

#Final dataframe

In [ ]:
df_final=df.drop(['biography','caption', 'profile_name', 'tokens', 'biography_wo_punc',	'tokens_wo_punc',	'caption_wo_punc'	,'biography_wo_stop',	'caption_wo_stop',	'profile_wo_punc'	,'profile_wo_stop',	'tokens_wo_stop'	,'biography_wo_stopfreq',	'caption_wo_stopfreq'	,'profile_wo_stopfreq',	'tokens_wo_stopfreq',	'biography_wo_stopfreqrare'	,'caption_wo_stopfreqrare',	'profile_wo_stopfreqrare'	,'tokens_wo_stopfreqrare','biography_stemmed'	,'caption_stemmed',	'tokens_stemmed'], axis=1)
df_final

In [ ]:
df_final